In [1]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras
import pathlib


In [2]:
#Load and prepare data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [3]:
#Create model
def create_model():
    model = keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
        keras.layers.Dropout(.2),
        keras.layers.Dense(10, activation=tf.nn.softmax)        
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=['accuracy']
    )
    
    return model

In [4]:
#Path for modelcheckpoint
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#Create Checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1
                                                 )

model = create_model()
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback]
         )

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 508us/step - loss: 1.1580 - acc: 0.6690 - val_loss: 0.7077 - val_acc: 0.7820

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
1000/1000 [==============================] - 0s 297us/step - loss: 0.4220 - acc: 0.8900 - val_loss: 0.5657 - val_acc: 0.8210

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
1000/1000 [==============================] - 0s 308us/step - loss: 0.2919 - acc: 0.9270 - val_loss: 0.4742 - val_acc: 0.8500

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
1000/1000 [==============================] - 0s 312us/step - loss: 0.2026 - acc: 0.9550 - val_loss: 0.4621 - val_acc: 0.8480

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
1000/1000 [==============================] - 0s 315us/step - loss: 0.1518 - acc: 0.9710 - val_loss: 0.4262 - val_acc: 0.8610

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
1000/1000 [=====

In [5]:
#Check checkpoint file
!dir {checkpoint_dir}

 Volume in drive C is OS
 Volume Serial Number is 14D4-50C0

 Directory of C:\Users\yuranan\ml\Tensorflow\training_1

18/07/13  01:56 PM    <DIR>          .
18/07/13  01:56 PM    <DIR>          ..
18/07/13  01:56 PM                71 checkpoint
18/07/13  01:56 PM         1,631,706 cp.ckpt.data-00000-of-00001
18/07/13  01:56 PM               647 cp.ckpt.index
               3 File(s)      1,632,424 bytes
               2 Dir(s)  280,621,821,952 bytes free


In [6]:
#Recreate model architecture
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, acc: {:5.2f}%".format(acc * 100))

1000/1000 [==============================] - 0s 96us/step
Untrained model, acc: 10.90%


In [7]:
#Load Weight from checkpoint
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print("Trained model, acc: {:5.2f}%".format(acc * 100))

1000/1000 [==============================] - 0s 43us/step
Trained model, acc: 88.00%


In [8]:
#Create checkpoint every 5 epochs
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 verbose=1,
                                                 save_weights_only=True,
                                                 period=5
                                                )
model = create_model()
model.fit(
    train_images,
    train_labels,
    epochs=50,
    validation_data=(test_images, test_labels),
    verbose=0,
    callbacks=[cp_callback]
)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [9]:

#Sort checkpoint files
checkpoints = pathlib.Path(checkpoint_dir).glob("*.index")
checkpoints = sorted(checkpoints, key=lambda cp: cp.stat().st_mtime)
checkpoints = [cp.with_suffix('') for cp in checkpoints]
lastest = str(checkpoints[-1])
checkpoints
lastest

[WindowsPath('training_2/cp-0030.ckpt'),
 WindowsPath('training_2/cp-0035.ckpt'),
 WindowsPath('training_2/cp-0040.ckpt'),
 WindowsPath('training_2/cp-0045.ckpt'),
 WindowsPath('training_2/cp-0050.ckpt')]

'training_2\\cp-0050.ckpt'

In [11]:
#Recreate lastest trained model
model = create_model()
model.load_weights(lastest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, acc: {:5.2f}%".format(acc * 100))

1000/1000 [==============================] - 0s 82us/step
Restored model, acc: 87.50%


In [13]:
#Manual save model
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels)
print("Manually save model, acc: {:5.2f}%".format(acc * 100))

1000/1000 [==============================] - 0s 94us/step
Manually save model, acc: 87.50%


In [14]:
#Save entire model
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 475us/step - loss: 1.1885 - acc: 0.6700
Epoch 2/5
1000/1000 [==============================] - 0s 203us/step - loss: 0.4539 - acc: 0.8690
Epoch 3/5
1000/1000 [==============================] - 0s 250us/step - loss: 0.3171 - acc: 0.9170
Epoch 4/5
1000/1000 [==============================] - 0s 234us/step - loss: 0.2080 - acc: 0.9500
Epoch 5/5
1000/1000 [==============================] - 0s 234us/step - loss: 0.1634 - acc: 0.9580


In [15]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Load entire model, acc: {:5.2f}%".format(acc * 100))

1000/1000 [==============================] - 0s 117us/step
Load entire model, acc: 86.00%
